# Rectangular Plate Bending Element

This derivation follows the procedure given in Chapter 12 of "A First Course in the Finite Element Method, 4th Edition" by Daryl L. Logan. Logan's method has been improved upon to allow for orthotropic behavior. Closed form solutions for stiffness and load matrices are provided.

We'll start by importing a few Python libraries that are useful for symbolic math, and initializing "pretty" printing.

In [1]:
from sympy import symbols, Matrix, diff, integrate, simplify, factor, latex, init_printing
from IPython.display import display, Math
init_printing()

The plate width will be defined as $2b$, and the height will be $2c$ to be consistent with Figure 12-1. We'll set up some Sympy symbols to represent $b$ and $c$.

In [2]:
b, c = symbols('b, c')

The plate is defined by four nodes specified in counter-clockwise order: i, j, m, and n. The local x-axis runs from node i toward node j, and the local y-axis runs from node i toward node n. Next we'll define the element's local displacement vector, $[d]$, at each node. There are 3 degrees of freedom at each node: $w$, $\theta_x$, and $\theta_y$.

In [3]:
wi, theta_xi, theta_yi = symbols('w_i, theta_x_i, theta_yi')
wj, theta_xj, theta_yj = symbols('w_j, theta_xj, theta_yj')
wm, theta_xm, theta_ym = symbols('w_m, theta_xm, theta_ym')
wn, theta_xn, theta_yn = symbols('w_n, theta_xn, theta_yn')
d = Matrix([wi, theta_xi, theta_yi, wj, theta_xj, theta_yj, wm, theta_xm, theta_ym, wn, theta_xn, theta_yn])
display(Math('[d] = ' + latex(d)))

<IPython.core.display.Math object>

A 12-term polynomial displacement function will be assumed to define the out-of-plane displacement, w, at any point (x, y) in the plate's local coordinate system. The rotations about each axis are derivatives of this displacement:

In [4]:
a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12 = symbols('a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11, a_12')
x, y, w, theta_x, theta_y = symbols('x, y, w, theta_x, theta_y')

w = a1 + a2*x + a3*y + a4*x**2 + a5*x*y + a6*y**2 + a7*x**3 + a8*x**2*y + a9*x*y**2 + a10*y**3 + a11*x**3*y + a12*x*y**3
theta_x = diff(w, y)
theta_y = -diff(w, x)

display(Math('w = ' + latex(w)))
display(Math('\\theta_x = ' + latex(theta_x)))
display(Math('\\theta_y = ' + latex(theta_y)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The negative sign on $\frac{dw}{dx}$ is required to be consistent with the right hand rule. These equations can be rewritten in matrix form as follows:

$[\psi] = [P][a]$

where $[\psi]$ is shorthand for  $\begin{bmatrix} w \\ \theta_x \\ \theta_y \end{bmatrix}$ and $[P]$ is defined as follows:

In [5]:
P = Matrix([[1, x, y, x**2, x*y, y**2, x**3, x**2*y, x*y**2, y**3, x**3*y, x*y**3],
            [0, 0, 1, 0, x, 2*y, 0, x**2, 2*x*y, 3*y**2, x**3, 3*x*y**2],
            [0, -1, 0, -2*x, -y, 0, -3*x**2, -2*x*y, -y**2, 0, -3*x**2*y, -y**3]])

display(Math('P = ' + latex(P)))

<IPython.core.display.Math object>

This general equation for $[P]$ will be evaluated at each node to give us a larger set of equations:

$[d] = [C][a]$

where $[C]$ is merely $[P]$ evaluated at each node, and $[d]$ is correpsondingly $[\psi]$ at each node. Knowing that the plate width is $2b$ and the plate height is $2c$, we can obtain the matrix $[C]$.

In [6]:
C = Matrix([P, P, P, P])
C[0:3, 0:12] = C[0:3, 0:12].subs(x, 0).subs(y, 0)      # i-node @ x = 0, y = 0
C[3:6, 0:12] = C[3:6, 0:12].subs(x, 2*b).subs(y, 0)    # j-node @ x = 2b, y = 0
C[6:9, 0:12] = C[6:9, 0:12].subs(x, 2*b).subs(y, 2*c)  # m-node @ x = 2b, y = 2c
C[9:12, 0:12] = C[9:12, 0:12].subs(x, 0).subs(y, 2*c)  # n-node @ x = 0, y = 2c
display(Math('[C] = ' + latex(C)))
print(C)

<IPython.core.display.Math object>

Matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2*b, 0, 4*b**2, 0, 0, 8*b**3, 0, 0, 0, 0, 0], [0, 0, 1, 0, 2*b, 0, 0, 4*b**2, 0, 0, 8*b**3, 0], [0, -1, 0, -4*b, 0, 0, -12*b**2, 0, 0, 0, 0, 0], [1, 2*b, 2*c, 4*b**2, 4*b*c, 4*c**2, 8*b**3, 8*b**2*c, 8*b*c**2, 8*c**3, 16*b**3*c, 16*b*c**3], [0, 0, 1, 0, 2*b, 4*c, 0, 4*b**2, 8*b*c, 12*c**2, 8*b**3, 24*b*c**2], [0, -1, 0, -4*b, -2*c, 0, -12*b**2, -8*b*c, -4*c**2, 0, -24*b**2*c, -8*c**3], [1, 0, 2*c, 0, 0, 4*c**2, 0, 0, 0, 8*c**3, 0, 0], [0, 0, 1, 0, 0, 4*c, 0, 0, 0, 12*c**2, 0, 0], [0, -1, 0, 0, -2*c, 0, 0, 0, -4*c**2, 0, 0, -8*c**3]])


An important matrix that we will come back to later is the shape function matrix $[N]$, defined as:

$[N] = [P][C]^{-1}$

The closed form solution of $[N]$ for a rectangular plate is:

In [7]:
N = P*C.inv()
display(Math('[N] = ' + latex(simplify(N))))

<IPython.core.display.Math object>

We can now solve for the $[a]$ matrix in terms of the nodal displacements:

In [8]:
a = simplify(C.inv()*d)
display(Math('[a] = ' + latex(a)))

<IPython.core.display.Math object>

The next step is to define the curvature matrix:

$[\kappa] = \begin{bmatrix} -\frac{d^2w}{dx^2} \\ -\frac{d^2w}{dy^2} \\ -\frac{2d^2w}{dxdy} \end{bmatrix} = [Q][a]$

It should be recognized that $w/[a]$ is simply the first row of our $[P]$ matrix. Evaluating the derivatives in this expression gives $[Q]$ as follows:

In [9]:
Q = Matrix([-diff(diff(P[0, :], x), x),
            -diff(diff(P[0, :], y), y),
            -2*diff(diff(P[0, :], x), y)])
display(Math('[Q] = ' + latex(Q)))
print(Q)

<IPython.core.display.Math object>

Matrix([[0, 0, 0, -2, 0, 0, -6*x, -2*y, 0, 0, -6*x*y, 0], [0, 0, 0, 0, 0, -2, 0, 0, -2*x, -6*y, 0, -6*x*y], [0, 0, 0, 0, -2, 0, 0, -4*x, -4*y, 0, -6*x**2, -6*y**2]])


With $[Q]$ in hand we can now solve for the $[B]$ matrix which is essential for formulating the stiffness matrix $[k]$

In [10]:
B = simplify(Q*C.inv())
display(Math('[B] = ' + latex(B)))

<IPython.core.display.Math object>

Now we form the constitutive matrix for orthotropic materials, [D]. This matrix is analagous to the flexural stiffness of a beam EI.

In [11]:
Ex, Ey, nu_xy, nu_yx, G, t = symbols('E_x, E_y, \\nu_{xy}, \\nu_{yx}, G, t')

D = 1/(1 - nu_xy*nu_yx)*Matrix([[   Ex,    nu_yx*Ex,          0         ],
                                [nu_xy*Ey,    Ey,             0         ],
                                [   0,        0,     (1 - nu_xy*nu_yx)*G]])

display(Math('[D] = \\frac{t^3}{12}' + latex(D)))

print(t**3/12*D)

<IPython.core.display.Math object>

Matrix([[E_x*t**3/(12*(-\nu_{xy}*\nu_{yx} + 1)), E_x*\nu_{yx}*t**3/(12*(-\nu_{xy}*\nu_{yx} + 1)), 0], [E_y*\nu_{xy}*t**3/(12*(-\nu_{xy}*\nu_{yx} + 1)), E_y*t**3/(12*(-\nu_{xy}*\nu_{yx} + 1)), 0], [0, 0, G*t**3/12]])


Now we can calculate the stiffness matrix:

$[k] = \int_0^{2c} \int_0^{2b} [B]^T[D][B] dx dy$

In [12]:
k = simplify(integrate(integrate(B.T*D*B, (x, 0, 2*b)), (y, 0, 2*c)))

display(Math('[k] = \\frac{t^3}{12}' + latex(k)))

# Uncomment the line below for a version that can be copied/pasted
# Be sure to multiply by t**3/12
print(k)

<IPython.core.display.Math object>

Matrix([[(-E_x*\nu_{yx}*b**2*c**2/4 - E_x*c**4 - E_y*\nu_{xy}*b**2*c**2/4 - E_y*b**4 + 7*G*\nu_{xy}*\nu_{yx}*b**2*c**2/5 - 7*G*b**2*c**2/5)/(b**3*c**3*(\nu_{xy}*\nu_{yx} - 1)), (-E_x*\nu_{yx}*c**2/2 - E_y*b**2 + G*\nu_{xy}*\nu_{yx}*c**2/5 - G*c**2/5)/(b*c**2*(\nu_{xy}*\nu_{yx} - 1)), (E_x*c**2 + E_y*\nu_{xy}*b**2/2 - G*\nu_{xy}*\nu_{yx}*b**2/5 + G*b**2/5)/(b**2*c*(\nu_{xy}*\nu_{yx} - 1)), (5*E_x*\nu_{yx}*b**2*c**2 + 20*E_x*c**4 + 5*E_y*\nu_{xy}*b**2*c**2 - 10*E_y*b**4 - 28*G*\nu_{xy}*\nu_{yx}*b**2*c**2 + 28*G*b**2*c**2)/(20*b**3*c**3*(\nu_{xy}*\nu_{yx} - 1)), (E_x*\nu_{yx}*c**2/2 - E_y*b**2/2 - G*\nu_{xy}*\nu_{yx}*c**2/5 + G*c**2/5)/(b*c**2*(\nu_{xy}*\nu_{yx} - 1)), (5*E_x*c**2 - G*\nu_{xy}*\nu_{yx}*b**2 + G*b**2)/(5*b**2*c*(\nu_{xy}*\nu_{yx} - 1)), (-5*E_x*\nu_{yx}*b**2*c**2 + 10*E_x*c**4 - 5*E_y*\nu_{xy}*b**2*c**2 + 10*E_y*b**4 + 28*G*\nu_{xy}*\nu_{yx}*b**2*c**2 - 28*G*b**2*c**2)/(20*b**3*c**3*(\nu_{xy}*\nu_{yx} - 1)), (-E_y*b**2/2 - G*\nu_{xy}*\nu_{yx}*c**2/5 + G*c**2/5)/(b*c**2*(\n

The surface force matrix $[F_s]$ can be obtained from the shape function matrix. Since we're interested in the surface force matrix for uniform pressures in the direction of w,

In [13]:
q = symbols('q')

Fs = integrate(integrate(N[0, :].T*q, (x, 0, 2*b)), (y, 0, 2*c))

display(Math('[F_s] = 4qcb' + latex(Fs/(4*q*c*b))))

# Uncomment the line below for a version that can be copied/pasted
print(Fs)

<IPython.core.display.Math object>

Matrix([[b*c*q], [b*c**2*q/3], [-b**2*c*q/3], [b*c*q], [b*c**2*q/3], [b**2*c*q/3], [b*c*q], [-b*c**2*q/3], [b**2*c*q/3], [b*c*q], [-b*c**2*q/3], [-b**2*c*q/3]])


## Membrane Action

### Shape Functions

In [14]:
from sympy import factor, symbols
r, s = symbols('r, s')
h1 = factor(1/4*(1-r)*(1-s))
h2 = factor(1/4*(1+r)*(1-s))
h3 = factor(1/4*(1+r)*(1+s))
h4 = factor(1/4*(1-r)*(1+s))

In [15]:
x1, y1, x2, y2, x3, y3, x4, y4 = symbols('x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4')
x = h1*x1 + h2*x2 + h3*x3 + h4*x4
y = h1*y1 + h2*y2 + h3*y3 + h4*y4

In [16]:
J = Matrix([[diff(x, r), diff(y, r)],
            [diff(x, s), diff(y, s)]])

display(Math('J = (1/4)' + latex(factor(J*4))))
print(J*4)

<IPython.core.display.Math object>

Matrix([[x_1*(s - 1) - 1.0*x_2*(s - 1) + x_3*(s + 1) - 1.0*x_4*(s + 1), y_1*(s - 1) - 1.0*y_2*(s - 1) + y_3*(s + 1) - 1.0*y_4*(s + 1)], [x_1*(r - 1) - 1.0*x_2*(r + 1) + x_3*(r + 1) - 1.0*x_4*(r - 1), y_1*(r - 1) - 1.0*y_2*(r + 1) + y_3*(r + 1) - 1.0*y_4*(r - 1)]])


In [17]:
print(4*diff(h1, r))
print(4*diff(h2, r))
print(4*diff(h3, r))
print(4*diff(h4, r))

s - 1.0
1.0 - 1.0*s
s + 1.0
-1.0*s - 1.0
